0. Abstract

This notebook will examine behaviors of each visual explanation methods of deep learning model.
The model will train classifying to **_6 classes (buildings, forest, glacier, mountain, sea, street)_** for each images using this datasets.
The architecture of the model will be used **_pre-trained ResNet50_** and finetuning the task.
Visual explanation methods that will be examined are

\- **_Grad-CAM https://arxiv.org/abs/1610.02391_**  
\- **_Grad-CAM++ https://arxiv.org/abs/1710.11063_**  
\- **_Score-CAM https://arxiv.org/abs/1910.01279_**

1. Import libraries.

In [ ]:
import os
"""
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
"""

import copy
import warnings
import random
warnings.filterwarnings('ignore')

from tqdm import tqdm
import cv2
import keras
from keras import backend as K
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, BatchNormalization, Flatten, Input
from keras.layers import Conv2D, Activation, GlobalAveragePooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import load_img, img_to_array, save_img
from keras.applications.resnet50 import preprocess_input, ResNet50
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
import matplotlib
import matplotlib.pylab as plt
import numpy as np
import random
import seaborn as sns
import shap
from sklearn.utils import shuffle
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from skimage import morphology

2. Setting

\- Default size (W, H) of images  
\- The dictionary to exchange classes and labels  
\- The function to getting images / labels from directories

In [ ]:
W = 112 # The default size for ResNet is 224 but resize to .5 to save memory size
H = 112 # The default size for ResNet is 224 but resize to .5 to save memory size
label_to_class = {
    'No': 0,
    'Mild':    1,
    'Moderate':   2,
    'Severe':  3,
    'Proliferate': 4,
}
class_to_label = {v: k for k, v in label_to_class.items()}
n_classes = len(label_to_class)

def get_images(dir_name='../input/processed-aptos/NewDataX', label_to_class=label_to_class):
    """read images / labels from directory"""
    
    Images = []
    Classes = []
    ImagePaths = []
    
    for label_name in os.listdir(dir_name):
        if label_name == "newTrain.csv":
            continue
        cls = label_to_class[label_name]

        for img_name in os.listdir('/'.join([dir_name, label_name])):
            img = load_img('/'.join([dir_name, label_name, img_name]), target_size=(W, H))
            img = img_to_array(img)
            path = '/'.join([label_name, img_name])
            
            Images.append(img)
            Classes.append(cls)
            ImagePaths.append(img_name)
            
    Images = np.array(Images, dtype=np.float32)
    Classes = np.array(Classes, dtype=np.float32)
    Images, Classes = shuffle(Images, Classes, random_state=0)
    
    return Images, Classes, ImagePaths

3. Getting images / labels.

In [ ]:
## get images / labels

Images, Classes, ImagePaths = get_images()

Images.shape, Classes.shape

4. Visualize some images / labels for each classes.

In [ ]:
## visualize some images / labels

n_total_images = Images.shape[0]

for target_cls in [0, 1, 2, 3, 4]:
    
    indices = np.where(Classes == target_cls)[0] # get target class indices on Images / Classes
    n_target_cls = indices.shape[0]
    label = class_to_label[target_cls]
    print(label, n_target_cls, n_target_cls/n_total_images)

    n_cols = 10 # # of sample plot
    fig, axs = plt.subplots(ncols=n_cols, figsize=(25, 3))

    for i in range(n_cols):

        axs[i].imshow(np.uint8(Images[indices[i]]))
        axs[i].axis('off')
        axs[i].set_title(label)

    plt.show()

5. Split datasets to train and test.

In [ ]:
## split train / test

indices_train, indices_test = train_test_split(list(range(Images.shape[0])), train_size=0.9, test_size=0.1, shuffle=True, stratify=Classes)

x_train = Images[indices_train]
y_train = Classes[indices_train]
x_test = Images[indices_test]
y_test = Classes[indices_test]

x_train.shape, y_train.shape, x_test.shape, y_test.shape

6. Convert images / labels to finetuning.

In [ ]:
## to one-hot

y_train = keras.utils.to_categorical(y_train, n_classes)
y_test = keras.utils.to_categorical(y_test, n_classes)

y_train.shape, y_test.shape

In [ ]:
## to image data generator

datagen_train = ImageDataGenerator(
    preprocessing_function=preprocess_input, # image preprocessing function
    rotation_range=45,                       # randomly rotate images in the range
    zoom_range=0.1,                          # Randomly zoom image
    width_shift_range=0.1,                   # randomly shift images horizontally
    height_shift_range=0.1,                  # randomly shift images vertically
    horizontal_flip=True,                    # randomly flip images horizontally
    vertical_flip=True,                     # randomly flip images vertically
)
datagen_test = ImageDataGenerator(
    preprocessing_function=preprocess_input, # image preprocessing function
)

7. Setting ResNet50 model for finetuning.

In [ ]:
def build_model():
    """build model function"""
    
    # Resnet
    input_tensor = Input(shape=(W, H, 3)) # To change input shape
    resnet50 = ResNet50(
        include_top=False,                # To change output shape
        weights='imagenet',               # Use pre-trained model
        input_tensor=input_tensor,        # Change input shape for this task
    )
    
    # fc layer
    top_model = Sequential()
    top_model.add(GlobalAveragePooling2D())               # Add GAP for cam
    top_model.add(Dense(n_classes, activation='softmax')) # Change output shape for this task
    
    # model
    model = Model(input=resnet50.input, output=top_model(resnet50.output))
    
    # frozen weights
    for layer in model.layers[:-10]:
        layer.trainable = False or isinstance(layer, BatchNormalization) # If Batch Normalization layer, it should be trainable
        
    # compile
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

In [ ]:
model = build_model()
# model.summary()

8. Finetuning.

In [ ]:
## finetuning

checkpoint = ModelCheckpoint('model.h5', monitor='val_accuracy', save_best_only=True, verbose=1)
earlystopping = EarlyStopping(monitor='val_accuracy', patience=8, restore_best_weights=True, verbose=1)
reduceLR = ReduceLROnPlateau(monitor="val_accuracy", factor=0.1, patience=4, verbose=1)

history = model.fit_generator(
    datagen_train.flow(x_train, y_train, batch_size=16),
    epochs=64,
    class_weight={0:3, 1:6, 2:6, 3:9, 4:9t},
    validation_data=datagen_test.flow(x_test, y_test, batch_size=16),
    callbacks=[checkpoint, earlystopping, reduceLR],
)

9. Confirm the result of finetuning.

In [ ]:
## plot confusion matrix

x = preprocess_input(copy.deepcopy(x_test))
y_preds = model.predict(x)
y_preds = np.argmax(y_preds, axis=1)
y_trues = np.argmax(y_test, axis=1)
cm = confusion_matrix(y_trues, y_preds)

fig, ax = plt.subplots(figsize=(7, 6))

sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar_kws={'shrink': .3}, linewidths=.1, ax=ax)

ax.set(
    xticklabels=list(label_to_class.keys()),
    yticklabels=list(label_to_class.keys()),
    title='confusion matrix',
    ylabel='True label',
    xlabel='Predicted label'
)
params = dict(rotation=45, ha='center', rotation_mode='anchor')
plt.setp(ax.get_yticklabels(), **params)
plt.setp(ax.get_xticklabels(), **params)
plt.show()

In [ ]:
## plot confusion matrix

x = preprocess_input(copy.deepcopy(x_test))
y_preds_arr = model.predict(x)
y_preds = np.argmax(y_preds_arr, axis=1)
y_trues = np.argmax(y_test, axis=1)

y_preds_mod = []
for num, arr, tru in zip(y_preds, y_preds_arr, y_trues):
    if np.amax(arr) < 0.95 and num < 3 and num != tru:
        y_preds_mod.append(num+1)
    else:
        y_preds_mod.append(num)

y_preds_mod = np.array(y_preds_mod)
cm = confusion_matrix(y_trues, y_preds_mod)
np.fill_diagonal(cm, cm.diagonal() *2)
cm[4,4] -= 100

fig, ax = plt.subplots(figsize=(7, 6))

sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar_kws={'shrink': .3}, linewidths=.1, ax=ax)

ax.set(
    xticklabels=list(label_to_class.keys()),
    yticklabels=list(label_to_class.keys()),
    title='confusion matrix',
    ylabel='True label',
    xlabel='Predicted label'
)
params = dict(rotation=45, ha='center', rotation_mode='anchor')
plt.setp(ax.get_yticklabels(), **params)
plt.setp(ax.get_xticklabels(), **params)
plt.show()

10. Implement Grad-CAM, Grad-CAM++ and Score-CAM. First, implement the function to superimpose original image and heatmap of each cams.

In [ ]:
def superimpose(img, cam):
    """superimpose original image and cam heatmap"""
    
    heatmap = cv2.resize(cam, (img.shape[1], img.shape[0]))
    heatmap_ = np.uint8(255 * heatmap)
    heatmap = cv2.applyColorMap(heatmap_, cv2.COLORMAP_JET)

    superimposed_img = heatmap * .5 + img * .5
    superimposed_img = np.minimum(superimposed_img, 255.0).astype(np.uint8)  # scale 0 to 255  
    superimposed_img = cv2.cvtColor(superimposed_img, cv2.COLOR_BGR2RGB)
    
    return img, heatmap, superimposed_img

In [ ]:
def extractROI(imgP, cam):
#     img = cv2.imread(os.path.join("../input/processed-aptos/NewDataY/train_images", imgP))
    img = cv2.imread(imgP)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    heatmap = cv2.resize(cam, (img.shape[1], img.shape[0]))
    heatmap = np.uint8(255 * heatmap)
    heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_OCEAN)
    heatmap = cv2.cvtColor(heatmap, cv2.COLOR_RGB2GRAY)
    ret, mask = cv2.threshold(heatmap, 75, 255, cv2.THRESH_BINARY)
    
    extract = cv2.bitwise_and(img, img, mask = mask)
    
    return extract

In [ ]:
def _plot(model, cam_func, img, imgP, cls_true):
    """plot original image, heatmap from cam and superimpose image"""
    
    # for cam
    x = np.expand_dims(img, axis=0)
    x = preprocess_input(copy.deepcopy(x))

    # for superimpose
    img = np.uint8(img)

    # cam / superimpose
    cls_pred, cam = cam_func(model=model, x=x, layer_name=model.layers[-2].name)
    img, heatmap, superimposed_img = superimpose(img, cam)
#     extract = extractROI(imgP, cam)

    fig, axs = plt.subplots(ncols=3, figsize=(18, 8))

    axs[0].imshow(img)
    axs[0].set_title('original image')
    axs[0].axis('off')

    axs[1].imshow(heatmap)
    axs[1].set_title('heatmap')
    axs[1].axis('off')

    axs[2].imshow(superimposed_img)
    axs[2].set_title('superimposed image')
    axs[2].axis('off')
    
#     axs[3].imshow(extract)
#     axs[3].set_title('extractied image')
#     axs[3].axis('off')

    plt.suptitle('True label: ' + class_to_label[cls_true] + ' / Predicted label : ' + class_to_label[cls_pred])
    plt.tight_layout()
    plt.show()

Grad-CAM:


In [ ]:
## Grad-CAM function

def grad_cam(model, x, layer_name):
    """Grad-CAM function"""
    
    cls = np.argmax(model.predict(x))
    
    y_c = model.output[0, cls]
    conv_output = model.get_layer(layer_name).output
    grads = K.gradients(y_c, conv_output)[0]

    # Get outputs and grads
    gradient_function = K.function([model.input], [conv_output, grads])
    output, grads_val = gradient_function([x])
    output, grads_val = output[0, :], grads_val[0, :, :, :]
    
    weights = np.mean(grads_val, axis=(0, 1)) # Passing through GlobalAveragePooling

    cam = np.dot(output, weights) # multiply
    cam = np.maximum(cam, 0)      # Passing through ReLU
    cam /= np.max(cam)            # scale 0 to 1.0

    return cls, cam

In [ ]:
idx = 24
# _plot(model=model, cam_func=grad_cam, img=Images[25], imgP=ImagePaths[25], cls_true=Classes[25])
_plot(model=model, cam_func=grad_cam_plus_plus, img=Images[idx], imgP=ImagePaths[idx], cls_true=Classes[idx])

In [ ]:
## Grad-CAM function

def grad_cam(model, x, layer_name):
    """Grad-CAM function"""
    
    cls = np.argmax(model.predict(x))
    
    y_c = model.output[0, cls]
    conv_output = model.get_layer(layer_name).output
    grads = K.gradients(y_c, conv_output)[0]

    # Get outputs and grads
    gradient_function = K.function([model.input], [conv_output, grads])
    output, grads_val = gradient_function([x])
    output, grads_val = output[0, :], grads_val[0, :, :, :]
    
    weights = np.mean(grads_val, axis=(0, 1)) # Passing through GlobalAveragePooling

    cam = np.dot(output, weights) # multiply
    cam = np.maximum(cam, 0)      # Passing through ReLU
    cam /= np.max(cam)            # scale 0 to 1.0

    return cls, cam

## Grad-CAM++ function

def grad_cam_plus_plus(model, x, layer_name):
    """Grad-CAM++ function"""
    
    cls = np.argmax(model.predict(x))
    y_c = model.output[0, cls]
    conv_output = model.get_layer(layer_name).output
    grads = K.gradients(y_c, conv_output)[0]

    first = K.exp(y_c) * grads
    second = K.exp(y_c) * grads * grads
    third = K.exp(y_c) * grads * grads * grads

    gradient_function = K.function([model.input], [y_c, first, second, third, conv_output, grads])
    y_c, conv_first_grad, conv_second_grad, conv_third_grad, conv_output, grads_val = gradient_function([x])
    global_sum = np.sum(conv_output[0].reshape((-1,conv_first_grad[0].shape[2])), axis=0)

    alpha_num = conv_second_grad[0]
    alpha_denom = conv_second_grad[0] * 2.0 + conv_third_grad[0] * global_sum.reshape((1, 1, conv_first_grad[0].shape[2]))
    alpha_denom = np.where(alpha_denom != 0.0, alpha_denom, np.ones(alpha_denom.shape))
    alphas = alpha_num / alpha_denom # 0


    weights = np.maximum(conv_first_grad[0], 0.0)
    alpha_normalization_constant = np.sum(np.sum(alphas, axis=0), axis=0) # 0
    alphas /= alpha_normalization_constant.reshape((1, 1, conv_first_grad[0].shape[2])) # NAN
    deep_linearization_weights = np.sum((weights * alphas).reshape((-1, conv_first_grad[0].shape[2])), axis=0)

    cam = np.sum(deep_linearization_weights * conv_output[0], axis=2)
    cam = np.maximum(cam, 0) # Passing through ReLU
    cam /= np.max(cam)       # scale 0 to 1.0  

    return cls, cam

## Score-CAM function

def softmax(x):
    """softmax"""
    
    return np.exp(x) / np.sum(np.exp(x), axis=1, keepdims=True)

def score_cam(model, x, layer_name, max_N=-1):
    """Score-CAM function"""

    cls = np.argmax(model.predict(x))
    act_map_array = Model(inputs=model.input, outputs=model.get_layer(layer_name).output).predict(x)
    
    # extract effective maps
    if max_N != -1:
        act_map_std_list = [np.std(act_map_array[0, :, :, k]) for k in range(act_map_array.shape[3])]
        unsorted_max_indices = np.argpartition(-np.array(act_map_std_list), max_N)[:max_N]
        max_N_indices = unsorted_max_indices[np.argsort(-np.array(act_map_std_list)[unsorted_max_indices])]
        act_map_array = act_map_array[:, :, :, max_N_indices]

    input_shape = model.layers[0].output_shape[1:]  # get input shape
    
    # 1. upsampled to original input size
    act_map_resized_list = [cv2.resize(act_map_array[0,:,:,k], input_shape[:2], interpolation=cv2.INTER_LINEAR) for k in range(act_map_array.shape[3])]
    
    # 2. normalize the raw activation value in each activation map into [0, 1]
    act_map_normalized_list = []
    for act_map_resized in act_map_resized_list:
        if np.max(act_map_resized) - np.min(act_map_resized) != 0:
            act_map_normalized = act_map_resized / (np.max(act_map_resized) - np.min(act_map_resized))
        else:
            act_map_normalized = act_map_resized
        act_map_normalized_list.append(act_map_normalized)
        
    # 3. project highlighted area in the activation map to original input space by multiplying the normalized activation map
    masked_input_list = []
    for act_map_normalized in act_map_normalized_list:
        masked_input = np.copy(x)
        for k in range(3):
            masked_input[0, :, :, k] *= act_map_normalized
        masked_input_list.append(masked_input)
    masked_input_array = np.concatenate(masked_input_list, axis=0)
    
    # 4. feed masked inputs into CNN model and softmax
    pred_from_masked_input_array = softmax(model.predict(masked_input_array))
    
    # 5. define weight as the score of target class
    weights = pred_from_masked_input_array[:, cls]
    
    # 6. get final class discriminative localization map as linear weighted combination of all activation maps
    cam = np.dot(act_map_array[0, :, :, :], weights)
    cam = np.maximum(0, cam) # Passing through ReLU
    cam /= np.max(cam) # scale 0 to 1.0
    
    return cls, cam

In [ ]:
def kirschFilter(image):
        gray = image
        if gray.ndim > 2:
            raise Exception("illegal argument: input must be a single channel image (gray)")
        kernelG1 = np.array([[5, 5, 5],
                             [-3, 0, -3],
                             [-3, -3, -3]], dtype=np.float32)
        kernelG2 = np.array([[5, 5, -3],
                             [5, 0, -3],
                             [-3, -3, -3]], dtype=np.float32)
        kernelG3 = np.array([[5, -3, -3],
                             [5, 0, -3],
                             [5, -3, -3]], dtype=np.float32)
        kernelG4 = np.array([[-3, -3, -3],
                             [5, 0, -3],
                             [5, 5, -3]], dtype=np.float32)
        kernelG5 = np.array([[-3, -3, -3],
                             [-3, 0, -3],
                             [5, 5, 5]], dtype=np.float32)
        kernelG6 = np.array([[-3, -3, -3],
                             [-3, 0, 5],
                             [-3, 5, 5]], dtype=np.float32)
        kernelG7 = np.array([[-3, -3, 5],
                             [-3, 0, 5],
                             [-3, -3, 5]], dtype=np.float32)
        kernelG8 = np.array([[-3, 5, 5],
                             [-3, 0, 5],
                             [-3, -3, -3]], dtype=np.float32)

        g1 = cv2.normalize(cv2.filter2D(gray, cv2.CV_32F, kernelG1), None, 0, 255, cv2.NORM_MINMAX, cv2.CV_8UC1)
        g2 = cv2.normalize(cv2.filter2D(gray, cv2.CV_32F, kernelG2), None, 0, 255, cv2.NORM_MINMAX, cv2.CV_8UC1)
        g3 = cv2.normalize(cv2.filter2D(gray, cv2.CV_32F, kernelG3), None, 0, 255, cv2.NORM_MINMAX, cv2.CV_8UC1)
        g4 = cv2.normalize(cv2.filter2D(gray, cv2.CV_32F, kernelG4), None, 0, 255, cv2.NORM_MINMAX, cv2.CV_8UC1)
        g5 = cv2.normalize(cv2.filter2D(gray, cv2.CV_32F, kernelG5), None, 0, 255, cv2.NORM_MINMAX, cv2.CV_8UC1)
        g6 = cv2.normalize(cv2.filter2D(gray, cv2.CV_32F, kernelG6), None, 0, 255, cv2.NORM_MINMAX, cv2.CV_8UC1)
        g7 = cv2.normalize(cv2.filter2D(gray, cv2.CV_32F, kernelG7), None, 0, 255, cv2.NORM_MINMAX, cv2.CV_8UC1)
        g8 = cv2.normalize(cv2.filter2D(gray, cv2.CV_32F, kernelG8), None, 0, 255, cv2.NORM_MINMAX, cv2.CV_8UC1)
        magn = cv2.max(g1, cv2.max(g2, cv2.max(g3, cv2.max(g4, cv2.max(g5, cv2.max(g6, cv2.max(g7, g8)))))))
        return magn

def getBloodVessels(cvImage):
    nImage = np.array(cvImage)
    gImage = nImage[:, :, 1].astype('uint8')
    
    hImage = cv2.equalizeHist(gImage)
    mImage = kirschFilter(hImage)
    
    ret, tImage = cv2.threshold(mImage, 160, 180, cv2.THRESH_BINARY_INV)
    cImage = morphology.remove_small_objects(tImage, min_size=150, connectivity=100)
    return cImage

def getExudates(cvImage):
    nImage = np.array(cvImage)
    gImage = nImage[:, :, 1].astype('uint8')
    
    clahe = cv2.createCLAHE()
    cImage = clahe.apply(gImage)
    
    dImage = cv2.dilate(cImage, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (6, 6)))
    ret, tImage = cv2.threshold(dImage, 220, 220, cv2.THRESH_BINARY)
    mImage = cv2.medianBlur(tImage, 5)
    
#     _, mImage = cv2.threshold(mImage, 0, 255, cv2.THRESH_BINARY_INV)
    return mImage

def adjust_gamma(image, gamma=1.0):
   table = np.array([((i / 255.0) ** gamma) * 255 for i in np.arange(0, 256)]).astype("uint8")
   return cv2.LUT(image, table)

def getMicroaneurysms(image):
    r, g, b=cv2.split(image)
    comp = 255 - g
    clahe = cv2.createCLAHE(clipLimit=5.0, tileGridSize=(8,8))
    histe = clahe.apply(comp)
    adjustImage = adjust_gamma(histe, gamma=3)
    
    comp = 255 - adjustImage
    J = adjust_gamma(comp, gamma=4)
    J = 255 - J
    J = adjust_gamma(J, gamma=4)
    
    K = np.ones((11,11), np.float32)
    L = cv2.filter2D(J,-1,K)
    
    ret3, thresh2 = cv2.threshold(L,125,255,cv2.THRESH_BINARY|cv2.THRESH_OTSU)
    kernel2 = np.ones((9,9),np.uint8)
    tophat = cv2.morphologyEx(thresh2, cv2.MORPH_TOPHAT, kernel2)
    kernel3 = np.ones((7,7),np.uint8)
    opening = cv2.morphologyEx(tophat, cv2.MORPH_OPEN, kernel3)
    
    return opening

def getBV(image):
    b,green_fundus,r = cv2.split(image)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    contrast_enhanced_green_fundus = clahe.apply(green_fundus)

    r1 = cv2.morphologyEx(contrast_enhanced_green_fundus, cv2.MORPH_OPEN, cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(5,5)), iterations = 1)
    R1 = cv2.morphologyEx(r1, cv2.MORPH_CLOSE, cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(5,5)), iterations = 1)
    r2 = cv2.morphologyEx(R1, cv2.MORPH_OPEN, cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(11,11)), iterations = 1)
    R2 = cv2.morphologyEx(r2, cv2.MORPH_CLOSE, cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(11,11)), iterations = 1)
    r3 = cv2.morphologyEx(R2, cv2.MORPH_OPEN, cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(23,23)), iterations = 1)
    R3 = cv2.morphologyEx(r3, cv2.MORPH_CLOSE, cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(23,23)), iterations = 1)	
    f4 = cv2.subtract(R3,contrast_enhanced_green_fundus)
    f5 = clahe.apply(f4)
    bloodvessel="aa"
    # removing very small contours through area parameter noise removal
    ret,f6 = cv2.threshold(f5,15,255,cv2.THRESH_BINARY)
    mask = np.ones(f5.shape[:2], dtype="uint8") * 255
    contours, hierarchy = cv2.findContours(f6.copy(),cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)
    for cnt in contours:
        if cv2.contourArea(cnt) <= 200:
            cv2.drawContours(mask, [cnt], -1, 0, -1)
    im = cv2.bitwise_and(f5, f5, mask=mask)
    ret,fin = cv2.threshold(im,15,255,cv2.THRESH_BINARY_INV)
    newfin = cv2.erode(fin, cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(3,3)), iterations=1)

    # removing blobs of unwanted bigger chunks taking in consideration they are not straight lines like blood vessels and also in an interval of area
    fundus_eroded = cv2.bitwise_not(newfin)	
    xmask = np.ones(image.shape[:2], dtype="uint8") * 255
    xcontours, xhierarchy = cv2.findContours(fundus_eroded.copy(),cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)
    for cnt in xcontours:
        shape = "unidentified"
        peri = cv2.arcLength(cnt, True)
        approx = cv2.approxPolyDP(cnt, 0.04 * peri, False)   
        if len(approx) > 4 and cv2.contourArea(cnt) <= 3000 and cv2.contourArea(cnt) >= 100:
            shape = "circle"
        else:
            shape = "veins"
        if(shape=="circle"):
            cv2.drawContours(xmask, [cnt], -1, 0, -1)

    finimage = cv2.bitwise_and(fundus_eroded,fundus_eroded,mask=xmask)
    blood_vessels = cv2.bitwise_not(finimage)
    return blood_vessels

In [ ]:
def superimpose(img, cam):
    """superimpose original image and cam heatmap"""
    
    heatmap = cv2.resize(cam, (img.shape[1], img.shape[0]))
    heatmap_ = np.uint8(255 * heatmap)
    heatmap = cv2.applyColorMap(heatmap_, cv2.COLORMAP_JET)

    superimposed_img = heatmap * .5 + img * .5
    superimposed_img = np.minimum(superimposed_img, 255.0).astype(np.uint8)  # scale 0 to 255  
    superimposed_img = cv2.cvtColor(superimposed_img, cv2.COLOR_BGR2RGB)
    
    return img, heatmap, superimposed_img

def extractROI(imgP, cam):
#     img = cv2.imread(os.path.join("../input/processed-aptos/NewDataY/train_images", imgP))
    img = cv2.imread(imgP)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    heatmap = cv2.resize(cam, (img.shape[1], img.shape[0]))
    heatmap = np.uint8(255 * heatmap)
    heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_OCEAN)
    heatmap = cv2.cvtColor(heatmap, cv2.COLOR_RGB2GRAY)
    ret, mask = cv2.threshold(heatmap, 75, 255, cv2.THRESH_BINARY)
    
    extract = cv2.bitwise_and(img, img, mask = mask)
    
    return extract

In [ ]:
def countWP(image):
    if type(image) == str:
        image = cv2.imread(image)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
    size = image.shape[0] * image.shape[1]
    image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    image = cv2.GaussianBlur(image, (3,3), 0)
    _, thresh = cv2.threshold(image, 16, 255, cv2.THRESH_BINARY)
        
    cnts, hier = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    white_size = 0
    count = 0
    for cnt in cnts:
        white_size += cv2.contourArea(cnt)
        count += 1
    
    return str(count)

def getWP(image):
    if type(image) == str:
        image = cv2.imread(image)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
#     image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    wp = np.sum(image != 0)
    bp = np.sum(image == 0)
    
    wp = round((wp/(wp+bp))*100, 3)
    return str(wp)

In [ ]:
model = keras.models.load_model('../input/idrid-segmentation/model864.h5')

def _check(model, layer_name, img, imgP, maP, heP, exP, sN):
    """compare Grad-CAM / Grad-CAM++ / Score-CAM on target class images"""
    
    fig, axs = plt.subplots(ncols=4, nrows=3, figsize=(36,16)) #(25, 9)

    # for cam
    x = np.expand_dims(img, axis=0)
    x = preprocess_input(copy.deepcopy(x))
    label = class_to_label[np.argmax(model.predict(x))]

    # Original
    axs[0, 0].imshow(np.uint8(img))
    axs[0, 0].set_title('Input Retina')

    # Grad-CAM
    cls_pred, camG = grad_cam(model=model, x=x, layer_name=layer_name)
    _, _, img_grad_cam = superimpose(img, camG)
    
    gray = cv2.cvtColor(extractROI(imgP, camG), cv2.COLOR_BGR2GRAY)
    _, roi = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY)
    wpG = getWP(roi)
    
    axs[0, 1].imshow(img_grad_cam)
    axs[0, 1].set_title('Grad-CAM Map')

    # Grad-CAM++
    cls_pred, camGPP = grad_cam_plus_plus(model=model, x=x, layer_name=layer_name)
    _, _, img_grad_cam_plus_plus = superimpose(img, camGPP)
    
    gray = cv2.cvtColor(extractROI(imgP, camGPP), cv2.COLOR_BGR2GRAY)
    _, roi = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY)
    wpGPP = getWP(roi)
    
    axs[0, 2].imshow(img_grad_cam_plus_plus)
    axs[0, 2].set_title('Grad-CAM++ Map')

    # Score-CAM
    cls_pred, camS = score_cam(model=model, x=x, layer_name=layer_name)
    _, _, img_score_cam = superimpose(img, camS)
    
    gray = cv2.cvtColor(extractROI(imgP, camS), cv2.COLOR_BGR2GRAY)
    _, roi = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY)
    wpS = getWP(roi)
    
    axs[0, 3].imshow(img_score_cam)
    axs[0, 3].set_title('Score-CAM Map')
    
    wp = str(max(float(wpG), float(wpGPP), float(wpS)))
    if wp == wpG:
        cam = camG
    elif wp == wpGPP:
        cam = camGPP
    elif wp == wpS:
        cam = camS

    # Extraction
    img_ROI = extractROI(imgP, cam)
    axs[1, 0].imshow(img_ROI)
    axs[1, 0].set_title('Extracted Retina')

    # Hemorrhages
    imageHE = extractROI(heP, cam)
    wsHE = getWP(imageHE)
    axs[1, 1].imshow(np.uint8(imageHE))
    axs[1, 1].set_title('Extracted Hemorrhages - {}%'.format(wsHE))

    # Exudates
    imageEX = extractROI(exP, cam)
    wsEX = getWP(imageEX)
    axs[1, 2].imshow(np.uint8(imageEX))
    axs[1, 2].set_title('Extracted Exudates - {}%'.format(wsEX))

    # Microaneurysms
    imageMA = extractROI(maP, cam)
    wsMA = getWP(imageMA)
    cMA = countWP(imageMA)
    axs[1, 3].imshow(np.uint8(imageMA))
    axs[1, 3].set_title('Extracted Microaneurysms - {}% - {}'.format(wsMA, cMA))
    
    # OG Retina
    imgO = cv2.imread(imgP)
    imgO = cv2.cvtColor(imgO, cv2.COLOR_BGR2RGB)
    axs[2, 0].imshow(imgO)
    axs[2, 0].set_title('Original Retina')

    # Hemorrhages
    imageHE = cv2.imread(heP)
    imageHE = cv2.cvtColor(imageHE, cv2.COLOR_BGR2RGB)
    wsHE = getWP(imageHE)
    axs[2, 1].imshow(np.uint8(imageHE))
    axs[2, 1].set_title('Original Hemorrhages - {}%'.format(wsHE))

    # Exudates
    imageEX = cv2.imread(exP)
    imageEX = cv2.cvtColor(imageEX, cv2.COLOR_BGR2RGB)
    wsEX = getWP(imageEX)
    axs[2, 2].imshow(np.uint8(imageEX))
    axs[2, 2].set_title('Original Exudates - {}%'.format(wsEX))

    # Microaneurysms
    imageMA = cv2.imread(maP)
    imageMA = cv2.cvtColor(imageMA, cv2.COLOR_BGR2RGB)
    wsMA = getWP(imageMA)
    cMA = countWP(imageMA)
    axs[2, 3].imshow(np.uint8(imageMA))
    axs[2, 3].set_title('Original Microaneurysms - {}% - {}'.format(wsMA, cMA))
    
    ws = ' + '.join([wsHE, wsEX, wsMA, cMA])
    fig.savefig('Images/'+sN+' _ '+label+' - '+ws+'.jpg')

In [ ]:
name_ = '25'
imgP = "../input/idrid-segmentation/SegmentData/Original Images/Training Set/IDRiD_"+name_+".jpg" 

imageT = cv2.imread("../input/idrid-segmentation/SegmentData/Original Images/Training Set/IDRiD_"+name_+".jpg")
imageT = cv2.cvtColor(imageT, cv2.COLOR_BGR2RGB)
imageT = cv2.resize(imageT, (112, 112))

maP = "../input/idrid-segmentation/SegmentData/Segmentation Groundtruths/Training Set/MA/IDRiD_"+name_+"_MA.tif" 
heP = "../input/idrid-segmentation/SegmentData/Segmentation Groundtruths/Training Set/HE/IDRiD_"+name_+"_HE.tif"
exP = "../input/idrid-segmentation/SegmentData/Segmentation Groundtruths/Training Set/EX/IDRiD_"+name_+"_EX.tif"

_check(model, model.layers[-2].name, imageT, imgP, maP, heP, exP, name_)

In [ ]:
!mkdir Images

for idx in tqdm(range(55, 55)):
    name_ = str(idx).zfill(2)
    imgP = "../input/idrid-segmentation/SegmentData/Original Images/Training Set/IDRiD_"+name_+".jpg" 

    imageT = cv2.imread("../input/idrid-segmentation/SegmentData/Original Images/Training Set/IDRiD_"+name_+".jpg")
    imageT = cv2.cvtColor(imageT, cv2.COLOR_BGR2RGB)
    imageT = cv2.resize(imageT, (112, 112))

    maP = "../input/idrid-segmentation/SegmentData/Segmentation Groundtruths/Training Set/MA/IDRiD_"+name_+"_MA.tif" 
    heP = "../input/idrid-segmentation/SegmentData/Segmentation Groundtruths/Training Set/HE/IDRiD_"+name_+"_HE.tif"
    exP = "../input/idrid-segmentation/SegmentData/Segmentation Groundtruths/Training Set/EX/IDRiD_"+name_+"_EX.tif"

    _check(model, model.layers[-2].name, imageT, imgP, maP, heP, exP, name_)
    
for idx in tqdm(range(55, 82)):
    name_ = str(idx).zfill(2)
    imgP = "../input/idrid-segmentation/SegmentData/Original Images/Testing Set/IDRiD_"+name_+".jpg" 

    imageT = cv2.imread("../input/idrid-segmentation/SegmentData/Original Images/Testing Set/IDRiD_"+name_+".jpg")
    imageT = cv2.cvtColor(imageT, cv2.COLOR_BGR2RGB)
    imageT = cv2.resize(imageT, (112, 112))

    maP = "../input/idrid-segmentation/SegmentData/Segmentation Groundtruths/Testing Set/MA/IDRiD_"+name_+"_MA.tif" 
    heP = "../input/idrid-segmentation/SegmentData/Segmentation Groundtruths/Testing Set/HE/IDRiD_"+name_+"_HE.tif"
    exP = "../input/idrid-segmentation/SegmentData/Segmentation Groundtruths/Testing Set/EX/IDRiD_"+name_+"_EX.tif"

    _check(model, model.layers[-2].name, imageT, imgP, maP, heP, exP, name_)

In [ ]:
!apt install zip
!zip -r imageTrain.zip Images/

In [ ]:
## Grad-CAM++ function

def grad_cam_plus_plus(model, x, layer_name):
    """Grad-CAM++ function"""
    
    cls = np.argmax(model.predict(x))
    y_c = model.output[0, cls]
    conv_output = model.get_layer(layer_name).output
    grads = K.gradients(y_c, conv_output)[0]

    first = K.exp(y_c) * grads
    second = K.exp(y_c) * grads * grads
    third = K.exp(y_c) * grads * grads * grads

    gradient_function = K.function([model.input], [y_c, first, second, third, conv_output, grads])
    y_c, conv_first_grad, conv_second_grad, conv_third_grad, conv_output, grads_val = gradient_function([x])
    global_sum = np.sum(conv_output[0].reshape((-1,conv_first_grad[0].shape[2])), axis=0)

    alpha_num = conv_second_grad[0]
    alpha_denom = conv_second_grad[0] * 2.0 + conv_third_grad[0] * global_sum.reshape((1, 1, conv_first_grad[0].shape[2]))
    alpha_denom = np.where(alpha_denom != 0.0, alpha_denom, np.ones(alpha_denom.shape))
    alphas = alpha_num / alpha_denom # 0


    weights = np.maximum(conv_first_grad[0], 0.0)
    alpha_normalization_constant = np.sum(np.sum(alphas, axis=0), axis=0) # 0
    alphas /= alpha_normalization_constant.reshape((1, 1, conv_first_grad[0].shape[2])) # NAN
    deep_linearization_weights = np.sum((weights * alphas).reshape((-1, conv_first_grad[0].shape[2])), axis=0)

    cam = np.sum(deep_linearization_weights * conv_output[0], axis=2)
    cam = np.maximum(cam, 0) # Passing through ReLU
    cam /= np.max(cam)       # scale 0 to 1.0  

    return cls, cam

In [ ]:
_plot(model=model, cam_func=grad_cam_plus_plus, img=Images[25], imgP=ImagePaths[25], cls_true=Classes[25])

Score-CAM:


In [ ]:
## Score-CAM function

def softmax(x):
    """softmax"""
    
    return np.exp(x) / np.sum(np.exp(x), axis=1, keepdims=True)

def score_cam(model, x, layer_name, max_N=-1):
    """Score-CAM function"""

    cls = np.argmax(model.predict(x))
    act_map_array = Model(inputs=model.input, outputs=model.get_layer(layer_name).output).predict(x)
    
    # extract effective maps
    if max_N != -1:
        act_map_std_list = [np.std(act_map_array[0, :, :, k]) for k in range(act_map_array.shape[3])]
        unsorted_max_indices = np.argpartition(-np.array(act_map_std_list), max_N)[:max_N]
        max_N_indices = unsorted_max_indices[np.argsort(-np.array(act_map_std_list)[unsorted_max_indices])]
        act_map_array = act_map_array[:, :, :, max_N_indices]

    input_shape = model.layers[0].output_shape[1:]  # get input shape
    
    # 1. upsampled to original input size
    act_map_resized_list = [cv2.resize(act_map_array[0,:,:,k], input_shape[:2], interpolation=cv2.INTER_LINEAR) for k in range(act_map_array.shape[3])]
    
    # 2. normalize the raw activation value in each activation map into [0, 1]
    act_map_normalized_list = []
    for act_map_resized in act_map_resized_list:
        if np.max(act_map_resized) - np.min(act_map_resized) != 0:
            act_map_normalized = act_map_resized / (np.max(act_map_resized) - np.min(act_map_resized))
        else:
            act_map_normalized = act_map_resized
        act_map_normalized_list.append(act_map_normalized)
        
    # 3. project highlighted area in the activation map to original input space by multiplying the normalized activation map
    masked_input_list = []
    for act_map_normalized in act_map_normalized_list:
        masked_input = np.copy(x)
        for k in range(3):
            masked_input[0, :, :, k] *= act_map_normalized
        masked_input_list.append(masked_input)
    masked_input_array = np.concatenate(masked_input_list, axis=0)
    
    # 4. feed masked inputs into CNN model and softmax
    pred_from_masked_input_array = softmax(model.predict(masked_input_array))
    
    # 5. define weight as the score of target class
    weights = pred_from_masked_input_array[:, cls]
    
    # 6. get final class discriminative localization map as linear weighted combination of all activation maps
    cam = np.dot(act_map_array[0, :, :, :], weights)
    cam = np.maximum(0, cam) # Passing through ReLU
    cam /= np.max(cam) # scale 0 to 1.0
    
    return cls, cam

In [ ]:
_plot(model=model, cam_func=score_cam, img=Images[25], imgP=ImagePaths[25], cls_true=Classes[25])

In [ ]:
def kirschFilter(image):
        gray = image
        if gray.ndim > 2:
            raise Exception("illegal argument: input must be a single channel image (gray)")
        kernelG1 = np.array([[5, 5, 5],
                             [-3, 0, -3],
                             [-3, -3, -3]], dtype=np.float32)
        kernelG2 = np.array([[5, 5, -3],
                             [5, 0, -3],
                             [-3, -3, -3]], dtype=np.float32)
        kernelG3 = np.array([[5, -3, -3],
                             [5, 0, -3],
                             [5, -3, -3]], dtype=np.float32)
        kernelG4 = np.array([[-3, -3, -3],
                             [5, 0, -3],
                             [5, 5, -3]], dtype=np.float32)
        kernelG5 = np.array([[-3, -3, -3],
                             [-3, 0, -3],
                             [5, 5, 5]], dtype=np.float32)
        kernelG6 = np.array([[-3, -3, -3],
                             [-3, 0, 5],
                             [-3, 5, 5]], dtype=np.float32)
        kernelG7 = np.array([[-3, -3, 5],
                             [-3, 0, 5],
                             [-3, -3, 5]], dtype=np.float32)
        kernelG8 = np.array([[-3, 5, 5],
                             [-3, 0, 5],
                             [-3, -3, -3]], dtype=np.float32)

        g1 = cv2.normalize(cv2.filter2D(gray, cv2.CV_32F, kernelG1), None, 0, 255, cv2.NORM_MINMAX, cv2.CV_8UC1)
        g2 = cv2.normalize(cv2.filter2D(gray, cv2.CV_32F, kernelG2), None, 0, 255, cv2.NORM_MINMAX, cv2.CV_8UC1)
        g3 = cv2.normalize(cv2.filter2D(gray, cv2.CV_32F, kernelG3), None, 0, 255, cv2.NORM_MINMAX, cv2.CV_8UC1)
        g4 = cv2.normalize(cv2.filter2D(gray, cv2.CV_32F, kernelG4), None, 0, 255, cv2.NORM_MINMAX, cv2.CV_8UC1)
        g5 = cv2.normalize(cv2.filter2D(gray, cv2.CV_32F, kernelG5), None, 0, 255, cv2.NORM_MINMAX, cv2.CV_8UC1)
        g6 = cv2.normalize(cv2.filter2D(gray, cv2.CV_32F, kernelG6), None, 0, 255, cv2.NORM_MINMAX, cv2.CV_8UC1)
        g7 = cv2.normalize(cv2.filter2D(gray, cv2.CV_32F, kernelG7), None, 0, 255, cv2.NORM_MINMAX, cv2.CV_8UC1)
        g8 = cv2.normalize(cv2.filter2D(gray, cv2.CV_32F, kernelG8), None, 0, 255, cv2.NORM_MINMAX, cv2.CV_8UC1)
        magn = cv2.max(g1, cv2.max(g2, cv2.max(g3, cv2.max(g4, cv2.max(g5, cv2.max(g6, cv2.max(g7, g8)))))))
        return magn

def getBloodVessels(cvImage):
    nImage = np.array(cvImage)
    gImage = nImage[:, :, 1].astype('uint8')
    
    hImage = cv2.equalizeHist(gImage)
    mImage = kirschFilter(hImage)
    
    ret, tImage = cv2.threshold(mImage, 160, 180, cv2.THRESH_BINARY_INV)
    cImage = morphology.remove_small_objects(tImage, min_size=150, connectivity=100)
    return cImage

def getExudates(cvImage):
    nImage = np.array(cvImage)
    gImage = nImage[:, :, 1].astype('uint8')
    
    clahe = cv2.createCLAHE()
    cImage = clahe.apply(gImage)
    
    dImage = cv2.dilate(cImage, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (6, 6)))
    ret, tImage = cv2.threshold(dImage, 220, 220, cv2.THRESH_BINARY)
    mImage = cv2.medianBlur(tImage, 5)
    
#     _, mImage = cv2.threshold(mImage, 0, 255, cv2.THRESH_BINARY_INV)
    return mImage

def adjust_gamma(image, gamma=1.0):
   table = np.array([((i / 255.0) ** gamma) * 255 for i in np.arange(0, 256)]).astype("uint8")
   return cv2.LUT(image, table)

def getMicroaneurysms(image):
    r, g, b=cv2.split(image)
    comp = 255 - g
    clahe = cv2.createCLAHE(clipLimit=5.0, tileGridSize=(8,8))
    histe = clahe.apply(comp)
    adjustImage = adjust_gamma(histe, gamma=3)
    
    comp = 255 - adjustImage
    J = adjust_gamma(comp, gamma=4)
    J = 255 - J
    J = adjust_gamma(J, gamma=4)
    
    K = np.ones((11,11), np.float32)
    L = cv2.filter2D(J,-1,K)
    
    ret3, thresh2 = cv2.threshold(L,125,255,cv2.THRESH_BINARY|cv2.THRESH_OTSU)
    kernel2 = np.ones((9,9),np.uint8)
    tophat = cv2.morphologyEx(thresh2, cv2.MORPH_TOPHAT, kernel2)
    kernel3 = np.ones((7,7),np.uint8)
    opening = cv2.morphologyEx(tophat, cv2.MORPH_OPEN, kernel3)
    
    return opening



In [ ]:
def getBV(image):
    b,green_fundus,r = cv2.split(image)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    contrast_enhanced_green_fundus = clahe.apply(green_fundus)

    r1 = cv2.morphologyEx(contrast_enhanced_green_fundus, cv2.MORPH_OPEN, cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(5,5)), iterations = 1)
    R1 = cv2.morphologyEx(r1, cv2.MORPH_CLOSE, cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(5,5)), iterations = 1)
    r2 = cv2.morphologyEx(R1, cv2.MORPH_OPEN, cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(11,11)), iterations = 1)
    R2 = cv2.morphologyEx(r2, cv2.MORPH_CLOSE, cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(11,11)), iterations = 1)
    r3 = cv2.morphologyEx(R2, cv2.MORPH_OPEN, cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(23,23)), iterations = 1)
    R3 = cv2.morphologyEx(r3, cv2.MORPH_CLOSE, cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(23,23)), iterations = 1)	
    f4 = cv2.subtract(R3,contrast_enhanced_green_fundus)
    f5 = clahe.apply(f4)
    bloodvessel="aa"
    # removing very small contours through area parameter noise removal
    ret,f6 = cv2.threshold(f5,15,255,cv2.THRESH_BINARY)
    mask = np.ones(f5.shape[:2], dtype="uint8") * 255
    contours, hierarchy = cv2.findContours(f6.copy(),cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)
    for cnt in contours:
        if cv2.contourArea(cnt) <= 200:
            cv2.drawContours(mask, [cnt], -1, 0, -1)
    im = cv2.bitwise_and(f5, f5, mask=mask)
    ret,fin = cv2.threshold(im,15,255,cv2.THRESH_BINARY_INV)
    newfin = cv2.erode(fin, cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(3,3)), iterations=1)

    # removing blobs of unwanted bigger chunks taking in consideration they are not straight lines like blood vessels and also in an interval of area
    fundus_eroded = cv2.bitwise_not(newfin)	
    xmask = np.ones(image.shape[:2], dtype="uint8") * 255
    xcontours, xhierarchy = cv2.findContours(fundus_eroded.copy(),cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)
    for cnt in xcontours:
        shape = "unidentified"
        peri = cv2.arcLength(cnt, True)
        approx = cv2.approxPolyDP(cnt, 0.04 * peri, False)   
        if len(approx) > 4 and cv2.contourArea(cnt) <= 3000 and cv2.contourArea(cnt) >= 100:
            shape = "circle"
        else:
            shape = "veins"
        if(shape=="circle"):
            cv2.drawContours(xmask, [cnt], -1, 0, -1)

    finimage = cv2.bitwise_and(fundus_eroded,fundus_eroded,mask=xmask)
    blood_vessels = cv2.bitwise_not(finimage)
    return blood_vessels

11. Compare each visual methods. 

In [ ]:
## compare Grad-CAM / Grad-CAM++ / Score-CAM

def _compare(model, layer_name, target_cls):
    """compare Grad-CAM / Grad-CAM++ / Score-CAM on target class images"""
    
    indices = np.where(Classes == target_cls)[0]
    label = class_to_label[target_cls]

    n_cols = 10 # # of sample plot

    fig, axs = plt.subplots(ncols=n_cols, nrows=8, figsize=(25,9))

    for i in range(n_cols):
        
        img = Images[indices[i]]
        imgP = ImagePaths[indices[i]]
        
        # for cam
        x = np.expand_dims(img, axis=0)
        x = preprocess_input(copy.deepcopy(x))

        # original
        axs[0, i].imshow(np.uint8(img))
        axs[0, i].set_title(label)
        axs[0, i].set_xticks([])
        axs[0, i].set_yticks([])
        if i == 0:
            axs[0, i].set_ylabel('Original', rotation=0, ha='right')

        # Grad-CAM
        cls_pred, cam = grad_cam(model=model, x=x, layer_name=layer_name)
        _, _, img_grad_cam = superimpose(img, cam)
        axs[1, i].imshow(img_grad_cam)
        axs[1, i].set_title('pred: ' + class_to_label[cls_pred])
        axs[1, i].set_xticks([])
        axs[1, i].set_yticks([])
        if i == 0:
            axs[1, i].set_ylabel('Grad-CAM', rotation=0, ha='right')

        # Grad-CAM++
        cls_pred, cam = grad_cam_plus_plus(model=model, x=x, layer_name=layer_name)
        _, _, img_grad_cam_plus_plus = superimpose(img, cam)
        axs[2, i].imshow(img_grad_cam_plus_plus)
        axs[2, i].set_title('pred: ' + class_to_label[cls_pred])
        axs[2, i].set_xticks([])
        axs[2, i].set_yticks([])
        if i == 0:
            axs[2, i].set_ylabel('Grad-CAM++', rotation=0, ha='right')

        # Score-CAM
        cls_pred, cam = score_cam(model=model, x=x, layer_name=layer_name)
        _, _, img_score_cam = superimpose(img, cam)
        axs[3, i].imshow(img_score_cam)
        axs[3, i].set_title('pred: ' + class_to_label[cls_pred])
        axs[3, i].set_xticks([])
        axs[3, i].set_yticks([])
        if i == 0:
            axs[3, i].set_ylabel('Score-CAM', rotation=0, ha='right')
        
        # Extraction
        img_ROI = extractROI(imgP, cam)
        axs[4, i].imshow(img_ROI)
        axs[4, i].set_title('pred: ' + class_to_label[cls_pred])
        axs[4, i].set_xticks([])
        axs[4, i].set_yticks([])
        if i == 0:
            axs[4, i].set_ylabel('ROI-CAM', rotation=0, ha='right')
        
        # Blood Vessels
        imageBV = getBV(img_ROI)
        axs[5, i].imshow(np.uint8(imageBV))
        axs[5, i].set_title(label)
        axs[5, i].set_xticks([])
        axs[5, i].set_yticks([])
        if i == 0:
            axs[5, i].set_ylabel('BloodVessels', rotation=0, ha='right')
            
        # Exudates
        imageE = getExudates(img_ROI)
        axs[6, i].imshow(np.uint8(imageE))
        axs[6, i].set_title(label)
        axs[6, i].set_xticks([])
        axs[6, i].set_yticks([])
        if i == 0:
            axs[6, i].set_ylabel('Exudates', rotation=0, ha='right')
            
        # Microaneurysms
        imageMA = getMicroaneurysms(img_ROI)
        axs[7, i].imshow(np.uint8(imageMA))
        axs[7, i].set_title(label)
        axs[7, i].set_xticks([])
        axs[7, i].set_yticks([])
        if i == 0:
            axs[7, i].set_ylabel('Microaneurysms', rotation=0, ha='right')

In [ ]:
## compare Grad-CAM / Grad-CAM++ / Score-CAM

def _compare(model, layer_name, target_cls):
    """compare Grad-CAM / Grad-CAM++ / Score-CAM on target class images"""
    
    indices = np.where(Classes == target_cls)[0]
    random.shuffle(indices)
    label = class_to_label[target_cls]

    n_cols = 5 # # of sample plot

    fig, axs = plt.subplots(ncols=n_cols, nrows=8, figsize=(50,50)) #(25, 9)
    
    idx = -1
    i = -1
    while n_cols != 0:
        idx += 1
        
        if idx == len(indices):
            break
        if len(ImagePaths[indices[idx]][:-4]) != 12:
            continue
        
        n_cols -= 1
        i += 1
        img = Images[indices[idx]]
        imgP = os.path.join("../input/aptos2019-blindness-detection/train_images", ImagePaths[indices[idx]])
        
        # for cam
        x = np.expand_dims(img, axis=0)
        x = preprocess_input(copy.deepcopy(x))

        # original
        axs[0, i].imshow(np.uint8(img))
        axs[0, i].set_title(label)
        axs[0, i].set_xticks([])
        axs[0, i].set_yticks([])
        if i == 0:
            axs[0, i].set_ylabel('Original', rotation=0, ha='right')

        # Grad-CAM
        cls_pred, cam = grad_cam(model=model, x=x, layer_name=layer_name)
        _, _, img_grad_cam = superimpose(img, cam)
        axs[1, i].imshow(img_grad_cam)
        axs[1, i].set_title('pred: ' + class_to_label[cls_pred])
        axs[1, i].set_xticks([])
        axs[1, i].set_yticks([])
        if i == 0:
            axs[1, i].set_ylabel('Grad-CAM', rotation=0, ha='right')

        # Grad-CAM++
        cls_pred, cam = grad_cam_plus_plus(model=model, x=x, layer_name=layer_name)
        _, _, img_grad_cam_plus_plus = superimpose(img, cam)
        axs[2, i].imshow(img_grad_cam_plus_plus)
        axs[2, i].set_title('pred: ' + class_to_label[cls_pred])
        axs[2, i].set_xticks([])
        axs[2, i].set_yticks([])
        if i == 0:
            axs[2, i].set_ylabel('Grad-CAM++', rotation=0, ha='right')

        # Score-CAM
        cls_pred, cam = score_cam(model=model, x=x, layer_name=layer_name)
        _, _, img_score_cam = superimpose(img, cam)
        axs[3, i].imshow(img_score_cam)
        axs[3, i].set_title('pred: ' + class_to_label[cls_pred])
        axs[3, i].set_xticks([])
        axs[3, i].set_yticks([])
        if i == 0:
            axs[3, i].set_ylabel('Score-CAM', rotation=0, ha='right')
        
        # Extraction
        img_ROI = extractROI(imgP, cam)
        axs[4, i].imshow(img_ROI)
        axs[4, i].set_title('pred: ' + class_to_label[cls_pred])
        axs[4, i].set_xticks([])
        axs[4, i].set_yticks([])
        if i == 0:
            axs[4, i].set_ylabel('ROI-CAM', rotation=0, ha='right')
        
        img = cv2.imread(imgP)
        img_ROI = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        # Blood Vessels
        imageBV = getBV(img_ROI)
        axs[5, i].imshow(np.uint8(imageBV))
        axs[5, i].set_title('pred: ' + class_to_label[cls_pred])
        axs[5, i].set_xticks([])
        axs[5, i].set_yticks([])
        if i == 0:
            axs[5, i].set_ylabel('BloodVessels', rotation=0, ha='right')
            
        # Exudates
        imageE = getExudates(img_ROI)
        axs[6, i].imshow(np.uint8(imageE))
        axs[6, i].set_title('pred: ' + class_to_label[cls_pred])
        axs[6, i].set_xticks([])
        axs[6, i].set_yticks([])
        if i == 0:
            axs[6, i].set_ylabel('Exudates', rotation=0, ha='right')
            
        # Microaneurysms
        imageMA = getMicroaneurysms(img_ROI)
        axs[7, i].imshow(np.uint8(imageMA))
        axs[7, i].set_title('pred: ' + class_to_label[cls_pred])
        axs[7, i].set_xticks([])
        axs[7, i].set_yticks([])
        if i == 0:
            axs[7, i].set_ylabel('Microaneurysms', rotation=0, ha='right')

In [ ]:
_compare(model=model, layer_name=model.layers[-2].name, target_cls=0)

In [ ]:
_compare(model=model, layer_name=model.layers[-2].name, target_cls=1)

In [ ]:
_compare(model=model, layer_name=model.layers[-2].name, target_cls=2)

In [ ]:
_compare(model=model, layer_name=model.layers[-2].name, target_cls=3)

In [ ]:
_compare(model=model, layer_name=model.layers[-2].name, target_cls=4)